In [111]:
DATE = '11'

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

# Load credentials from JSON file - MAKE SURE TO MAKE ONE FIRST, ENABLE GOOGLE DRIVE AND GOOGLE SHEETS API (ask chatgpt)
# Replace the form-response-resume-extracter thing with your credentials json file to use it
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("form-response-resume-extracter.json", scopes=SCOPES)

# Authorize gspread
client = gspread.authorize(creds)

# Open the Google Sheet
SPREADSHEET_ID = "1cohjNpgtIBmoFyOwBW16pe1oryK4G22NALlSQ7BTByY"  # Replace with actual ID from the sheet URL
sheet = client.open_by_key(SPREADSHEET_ID).sheet1

# Get all data from the applications sheet
data = sheet.get_all_records()

In [113]:
import pandas as pd
# Open the Google Sheet for people selected to quals
SPREADSHEET_ID = "1dtldMuoF3UcdEcYdzDW2isWl-Uex0Ws1h0xtMBMgtRg"  # Replace with actual ID from the sheet URL
sheet = client.open_by_key(SPREADSHEET_ID).sheet1

# Get all data from the qualifiers sheet
quals_df = pd.DataFrame(sheet.get_all_records())
quals_df_filtered = quals_df[quals_df['2/'+DATE] == 'TRUE']
emails = quals_df_filtered['Email Address'].to_list()

In [114]:
df = pd.DataFrame(data)
df["Resume Name"] = "Resume_" + df['Full Name'].str.title().str.split().str[0] + "_" + df['Full Name'].str.title().str.split().str[-1] + "_" + df.index.astype(str) + ".pdf"
resume_key = "Please upload your resume here as a PDF. \n\nNote: by submitting your resume here, you are consenting to having it be shared with our sponsors in the event that your application is accepted."
# Assuming the sheet has 'Email' and 'PDF Link' columns
filtered_df = df[df["Email Address"].isin(emails)]  # Filter rows where email is in x
pdf_links = filtered_df.set_index('Resume Name')[resume_key].to_dict()

In [115]:
detected_emails = filtered_df['Email Address'].tolist()
manual_download = []
for email in emails:
    if email not in detected_emails:
        manual_download.append(email)

In [ ]:
delinquent_names = quals_df[quals_df['Email Address'].isin(manual_download)]['Full Name'].str.lower().tolist()
ok_names = []
df['First Last Name'] = df['Full Name'].str.lower().str.split().str[0] + " " + df['Full Name'].str.lower().str.split().str[-1]
dels_df = df[df["First Last Name"].isin(delinquent_names)]  # Filter rows where email is in x
for name in delinquent_names:
    if name in dels_df['First Last Name'].values:
        if dels_df["First Last Name"].value_counts()[name] == 1:
            ok_names.append(name)
        else:
            print(name, 'appears', dels_df["First Last Name"].value_counts()[name], 'times')
    else:
        print('ERROR NAME NOT FOUND:', name)

In [117]:
new_filtered_df = dels_df[dels_df["First Last Name"].isin(ok_names)]  # Filter rows where email is in x
new_pdf_links = new_filtered_df.set_index('Resume Name')[resume_key].to_dict()
pdf_links.update(new_pdf_links)

In [ ]:
import os
import requests
import re

def extract_file_id(url):
    match = re.search(r"(?:id=|/d/|uc\?id=)([a-zA-Z0-9_-]+)", url)
    return match.group(1) if match else None

# Create the Drive API service
from googleapiclient.discovery import build
drive_service = build("drive", "v3", credentials=creds)

# Folder to save PDFs
output_folder = f"feb{DATE}_quals_resumes"
os.makedirs(output_folder, exist_ok=True)

def download_pdf(file_id, save_path):
    """Download a PDF from Google Drive using file ID."""
    try:
        request = drive_service.files().get_media(fileId=file_id)
        with open(save_path, "wb") as f:
            f.write(request.execute())
        # print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Failed to download {file_id}: {e}")

for file_name, url in pdf_links.items():
    file_id = extract_file_id(url)
    if file_id:
        file_path = os.path.join(output_folder, file_name)
        try:
            download_pdf(file_id, file_path)
            print(f"Downloaded: {file_path}")
        except Exception as e:
            print(f"Failed to download {file_id}: {e}")
    else:
        print(f"Invalid URL: {url}")